### About this notebook: 
<font color='grey'>
</font>

In [ ]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats 
import numpy as np
import json
import statsmodels.stats.multitest as multi
import matplotlib.pyplot as plt
import math
sys.path.append('../scripts/')
import Docket_integration

### Define the input directory and the output directory: 
<font color='grey'> 
    "input_dir": directory of input data<br/>
    "output_dir":directory of output data<br/>
</font>

In [ ]:
directories = {"input_dir":"../data/Data_input_for_LUAD",
               "output_dir":"../Output_LUAD"}

### About the input file and the output files: 
<font color='grey'> 
    "input_file_1": the mutation matrix;<br/>
    "input_file_2": the expression matrix;<br/>
    "File_out": the merged file;<br/>
</font>

In [ ]:
input_data = {"input_file_1":"Mut_GDSC_matrix.csv",
"input_file_2":"Exp_GDSC_matrix.csv",
"output_file":"mut_expr_merge.csv",
}

In [ ]:
input_data2 = {"Mutlist": ["TP53"],
"Exprlist": [],
"Output":"mut_exp_pair.csv",
}

### Data processing --  Step 1: 
<font color='grey'> 
    Read tables;<br/>
    Merge tables;<br/>
    
</font>

In [ ]:
Mut_mat = pd.read_csv(directories['input_dir'] + '/' + input_data['input_file_1'], index_col = "Unnamed: 0")
Expr_matrix = pd.read_csv(directories['input_dir'] + '/' + input_data['input_file_2'], index_col = "Unnamed: 0")

In [ ]:
label_for_merge = Docket_integration.matrix_comp(Mut_mat,Expr_matrix )   

In [ ]:
if label_for_merge[0] > 0.9:
    Merged_mat = Docket_integration.merge_matrix(Mut_mat, Expr_matrix, 'Row','Row')
elif label_for_merge[1] > 0.9:
    Merged_mat = Docket_integration.merge_matrix(Mut_mat, Expr_matrix, 'Col','Col')
    
Merged_mat.to_csv(directories['output_dir'] + '/' +input_data['output_file'])

### Data processing --  Step 2: 
<font color='grey'> 
    Integrating two datasets through statistical modeling;<br/>
</font>

In [ ]:
if len(input_data2['Mutlist']) > 0:
    Features_groupA = input_data2['Mutlist']
    Features_groupA = list(set(Features_groupA).intersection(set(list(Mut_mat.columns.values))))
else:
    Features_groupA = list(Mut_mat.columns.values)

if len(input_data2['Exprlist']) > 0:
    Features_groupB = input_data2['Exprlist']
    Features_groupB = list(set(Features_groupB).intersection(set(list(Expr_matrix.columns.values))))
else:
    Features_groupB = list(Expr_matrix.columns.values)


result = Docket_integration.Integration_category_numeric(Merged_mat, Features_groupA, Features_groupB)

In [ ]:
result.to_csv(directories['output_dir'] + '/' + input_data2['Output'])

In [ ]:
result.sort_values(by = ['p'])

In [ ]:
result.loc[result['FDR'] < 0.05].sort_values(by = ['FDR'])